In [1]:
import os
import sys
from tqdm import trange
from tqdm import tqdm
from skimage.util import montage
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

import medmnist
from medmnist.dataset import PathMNIST, ChestMNIST, DermaMNIST, OCTMNIST, PneumoniaMNIST, RetinaMNIST, BreastMNIST, OrganMNISTAxial, OrganMNISTCoronal, OrganMNISTSagittal
from medmnist.evaluator import getAUC, getACC
from medmnist.info import INFO

/root/miniconda3/envs/lora/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print("Version:", medmnist.__version__)

Version: 2.2.3


In [3]:
data_flag = 'dermamnist'
#data_flag = 'breastmnist'
download = True
input_root = "./"
NUM_EPOCHS = 10
BATCH_SIZE = 128
lr = 0.001

flag_to_class = {
    "pathmnist": PathMNIST,
    "chestmnist": ChestMNIST,
    "dermamnist": DermaMNIST,
    "octmnist": OCTMNIST,
    "pneumoniamnist": PneumoniaMNIST,
    "retinamnist": RetinaMNIST,
    "breastmnist": BreastMNIST,
    "organmnist_axial": OrganMNISTAxial,
    "organmnist_coronal": OrganMNISTCoronal,
    "organmnist_sagittal": OrganMNISTSagittal,
}

DataClass = flag_to_class[data_flag]

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

In [4]:
# preprocessing
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

# load the data
train_dataset = DataClass(root=input_root, split='train', transform=data_transform, download=download)
test_dataset = DataClass(root=input_root, split='test', transform=data_transform, download=download)
nonorm_dataset = DataClass(root=input_root, split='train', transform=transforms.ToTensor(), download=download)

# encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True)

Using downloaded and verified file: ./dermamnist.npz
Using downloaded and verified file: ./dermamnist.npz
Using downloaded and verified file: ./dermamnist.npz


In [5]:
# Data transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 调整图像大小为 (224, 224)
    transforms.ToTensor(),
])

# Load DermaMNIST dataset
train_dataset = DataClass(root=input_root, split='train', transform=transform, download=download)
test_dataset = DataClass(root=input_root, split='test', transform=transform, download=download)
nonorm_dataset = DataClass(root=input_root, split='train', transform=transforms.ToTensor(), download=download)
# encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=True)

Using downloaded and verified file: ./dermamnist.npz
Using downloaded and verified file: ./dermamnist.npz
Using downloaded and verified file: ./dermamnist.npz


In [16]:
# AlexNet Model
class AlexNet(nn.Module):
    def __init__(self, num_classes=7):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x = self.classifier(x)
        return x


In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
from torchvision import transforms
from tqdm import tqdm
from sklearn.metrics import accuracy_score, roc_auc_score
import torch.nn.functional as F

# 模型、损失和优化器
model = AlexNet(num_classes=7)
model = nn.DataParallel(model)  # 使用DataParallel进行多GPU训练
model = model.to('cuda')  # 将模型移动到GPU上
criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 训练和验证
NUM_EPOCHS = 100
for epoch in range(NUM_EPOCHS):
    train_correct = 0
    train_total = 0
    test_correct = 0
    test_total = 0
    all_true_labels = []
    all_predicted_probs = []

    # 训练
    model.train()
    for inputs, targets in tqdm(train_loader):
        inputs, targets = inputs.to('cuda'), targets.to('cuda')
        optimizer.zero_grad()
        outputs = model(inputs)
        targets = targets.squeeze().long()
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        train_total += targets.size(0)
        train_correct += (predicted == targets).sum().item()
    
    # 验证
    model.eval()
    all_preds = []
    all_targets = []
    
    with torch.no_grad():
        for inputs, targets in tqdm(test_loader):
            inputs, targets = inputs.to('cuda'), targets.to('cuda')
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_targets.extend(targets.cpu().numpy())
            # 添加真实标签和预测概率
            all_true_labels.extend(targets.cpu().numpy())
            all_predicted_probs.extend(F.softmax(outputs, dim=1).cpu().numpy())

    # 计算准确度和AUC
    correct = 0
    total = 0
    for i, j in zip(all_preds, all_targets):
        if i == j:
            correct += 1
        total += 1
    accuracy = correct / total
    
    # 计算 AUC
    auc = roc_auc_score(all_true_labels, all_predicted_probs, multi_class='ovr')

    print(f'Epoch {epoch + 1}/{NUM_EPOCHS}:')
    print(f'Training Accuracy: {train_correct / train_total:.4f}')
    print(f'Validation Accuracy: {accuracy:.4f}')
    print(f'Validation AUC: {auc:.4f}')


100%|██████████| 16/16 [00:02<00:00,  5.44it/s]


Epoch 1/100:
Training Accuracy: 0.6003
Validation Accuracy: 0.6688
Validation AUC: 0.4483


100%|██████████| 16/16 [00:02<00:00,  5.57it/s]


Epoch 2/100:
Training Accuracy: 0.6698
Validation Accuracy: 0.6688
Validation AUC: 0.4258


100%|██████████| 16/16 [00:02<00:00,  5.45it/s]


...

Epoch 98/100:
Training Accuracy: 0.6867
Validation Accuracy: 0.5930
Validation AUC: 0.8398


100%|██████████| 16/16 [00:03<00:00,  5.18it/s]


Epoch 99/100:
Training Accuracy: 0.6899
Validation Accuracy: 0.5756
Validation AUC: 0.8380


100%|██████████| 16/16 [00:03<00:00,  5.20it/s]

Epoch 100/100:
Training Accuracy: 0.6919
Validation Accuracy: 0.5825
Validation AUC: 0.8390
